In [4]:
import os
%cd ../../..
print(os.getcwd())

/Users/daniel.vella/Desktop/dv-box/msc/deepchem
/Users/daniel.vella/Desktop/dv-box/msc/deepchem


In [5]:
import torch

In [105]:
"""
Train low-data siamese models on MUV. Test last fold only.
"""
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import numpy as np
import deepchem as dc
import tensorflow as tf
import pandas as pd
import copy

In [208]:
tasks, datasets, transformers = dc.molnet.load_sider(featurizer='ECFP', splitter='random')
train_dataset, valid_dataset, test_dataset = datasets

In [240]:
def get_task_datasets(dataset):
    datasets = []
    num_tasks = len(dataset.tasks)
    
    for n in range(num_tasks):        
        X = pd.DataFrame(dataset.X, columns = ['X' + str(i + 1) for i in range(dataset.X.shape[1])])
        y = pd.Series(dataset.y[:, n], name='y')
        w = pd.Series(dataset.w[:, n], name='w')
        ids = pd.Series(dataset.ids, name='ids')
    
        df = X.assign(y=y.values, w=w.values, ids=ids.values, task=dataset.tasks[n])
        ds = dc.data.DiskDataset.from_dataframe(df)
        datasets.append(ds)
    return datasets

train_datasets = get_task_datasets(train_dataset)
test_datasets = get_task_datasets(test_dataset)

In [241]:
pytorch_model = torch.nn.Sequential(
    torch.nn.Linear(1024, 1000),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1000, 1)
)

model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

# metric = dc.metrics.Metric(dc.metrics.roc_auc_score, mode="classification")
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

model.fit(train_datasets[0], nb_epoch=200)

0.010704271793365479

In [242]:
print('training set score:', model.evaluate(train_datasets[0], [metric]))
print('test set score:', model.evaluate(test_datasets[0], [metric]))

training set score: {'pearson_r2_score': 0.9935988886688238}
test set score: {'pearson_r2_score': 0.1778891014960177}


In [ ]:
# task_splitter = dc.splits.TaskSplitter()
# fold_datasets = task_splitter.k_fold_split(dataset, K)

# train_folds = fold_datasets[:-1]
# train_dataset = dc.splits.merge_fold_datasets(train_folds)
# test_dataset = fold_datasets[-1]

# Train support model on train
# support_model = dc.nn.SequentialSupportGraph(n_feat)

# # Add layers
# support_model.add(dc.nn.GraphConv(64, n_feat, activation='relu'))
# support_model.add(dc.nn.GraphPool())
# support_model.add(dc.nn.GraphConv(128, 64, activation='relu'))
# support_model.add(dc.nn.GraphPool())
# support_model.add(dc.nn.GraphConv(64, 128, activation='relu'))
# support_model.add(dc.nn.GraphPool())
# support_model.add(dc.nn.Dense(128, 64, activation='tanh'))

# support_model.add_test(dc.nn.GraphGather(test_batch_size, activation='tanh'))
# support_model.add_support(
#     dc.nn.GraphGather(support_batch_size, activation='tanh'))

# model = dc.models.SupportGraphClassifier(
#     support_model,
#     test_batch_size=test_batch_size,
#     support_batch_size=support_batch_size,
#     learning_rate=learning_rate)

# model.fit(
#     train_dataset,
#     nb_epochs=nb_epochs,
#     n_episodes_per_epoch=n_train_trials,
#     n_pos=n_pos,
#     n_neg=n_neg,
#     log_every_n_samples=log_every_n_samples)
# mean_scores, std_scores = model.evaluate(
#     test_dataset, metric, n_pos, n_neg, n_trials=n_eval_trials)

# print("Mean Scores on evaluation dataset")
# print(mean_scores)
# print("Standard Deviations on evaluation dataset")
# print(std_scores)
# print("Median of Mean Scores")
# print(np.median(np.array(mean_scores.values())))
